<h1>Training do Modelo</h1>

<b> Author:</b><a href="https://github.com/CaiqueCaah"> Caique Gonçalves<a/>

<b>Author:</b><a href="https://github.com/doctorget"> Samuel Soares</a>

In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


<h3>Lendo o arquivo CSV</h3>

In [16]:
train = pd.read_csv("dataset_train.csv") 

<h3>Verificando a quantidade de saidas possiveis</h3>

In [5]:
len(train.label.value_counts())

2235

<p>Verificando a quantidade de saidas para escolha de ativação da rede neural.</p>

<h3>Verificando a quantidade de arquivos por pasta</h3>

In [5]:
train.label.value_counts()

3308    10
3442    10
3571    10
3752    10
5159    10
        ..
5176    10
3047    10
1984    10
5110    10
2862    10
Name: label, Length: 2235, dtype: int64

<p>Verificando se todas as pastas tem a quantidade de arquivos parecidos para que a relevancia seja igual.</p>

<h3>Carregamento da Imagem</h3>

In [9]:
train_image = []
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['path'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)

  0%|▏                                                                             | 154/67058 [00:05<37:17, 29.90it/s]


KeyboardInterrupt: 

<p>Convertendo a imagem para <i>array</i> e normalizando os pixels.</p>

In [17]:
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

(207, 224, 224, 3)

<p>Transformando em um <i>array numpy</i>.</p>

In [8]:
y = train['label']
X = train['path']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

<p>Coletando 20% dos dados para teste.</p>

<h3>Transformação dos dados</h3>

In [16]:
X_train = X_train.reshape(X_train.shape[0], 224, 224, 3)
X_test = X_test.reshape(X_test.shape[0], 224, 224, 3)

<p>Convertendo os dados para o formato do TensorFlow</p>

In [20]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

<p>Convertendo os dados para <i>float.</i></p>

In [21]:
X_train /= 255
X_test /= 255

<p>Aplicando a normalização para que fiquem entre 0 e 1.</p>

<h3>Criando os <i>Dummies</i></h3>

In [ ]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [23]:
y_train.head()

,1333,1543,1746,2352,2529,2556,2579,2628,2907,3101
76,0,0,0,1,0,0,0,0,0,0
140,0,0,0,0,0,0,1,0,0,0
203,0,0,0,0,0,0,0,0,0,1
147,0,0,0,0,0,0,1,0,0,0
190,0,0,0,0,0,0,0,0,1,0


<h3>Criando o Classificador</h3>
<h5>Operador de convulução.</h5>

In [24]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(224, 224, 3), activation = 'relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32, (3,3), activation = 'relu')) 

<p>Gerando os detectores de caracteristicas.</p>

In [ ]:
model.add(BatchNormalization()) 

<p>Aplicando a normalização para que fiquem entre 0 e 1 na camada de convolução (Mapa de caracteristicas).</p>

In [ ]:
model.add(MaxPooling2D(pool_size = (2,2)))

<p>Pooling.</p>
<p>Coletando os dados com maior valor.</p>

In [ ]:
model.add(Flatten())

<p>Flatten</p>
<p>Transforma a <i>matriz</i> em <i>array</i>.</p>

<h3>Criando a Rede Neural Densa</h3>

In [25]:
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 1024, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 1024, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 512, activation = 'relu')) 
model.add(Dropout(0.2))

In [26]:
model.add(Dense(units = len(train.label.value_counts()), activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

<p>Aplicando a função de ativação <i>softmax</i> na rede neural</p>

<h3>Treinando o modelo</h3>

In [ ]:
model.fit(X_train, y_train, batch_size = 128, epochs = 10, validation_data = (X_test, y_test))

<h3>Testando o modelo</h3>

In [33]:
resultado = model.evaluate(X_test, y_test)
resultado

42/42 [==============================] - 4s 87ms/step


[2.323156640643165, 0.190476194024086]